# 🧠 BERTOPIC

## 🧰 Librerías e importaciones
- Importa pandas para el flujo de tópicos.

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import torch
import pickle
import pandas as pd
from pathlib import Path
import json
import logging
import io

- El código está pensado para cargar y combinar archivos .pkl (pickles) que fueron guardados en máquinas con CUDA/GPU, pero forzando su lectura en CPU. Lo hace con varias “capas de rescate” por si la carga falla, y luego concatena los datos válidos en un único DataFrame final, generando además resúmenes y análisis.

In [ ]:
# FORZAR que PyTorch piense que CUDA no existe
torch.cuda.is_available = lambda: False

class DefinitiveCudaLoader:
    """Cargador definitivo que maneja archivos PKL con CUDA embedido"""
    
    def __init__(self, output_dir):
        self.output_dir = Path(output_dir)
        self.status_file = self.output_dir / 'processing_status.json'
        
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
        self.load_processing_status()
    
    def load_processing_status(self):
        if self.status_file.exists():
            with open(self.status_file, 'r') as f:
                self.status = json.load(f)
        else:
            self.status = {'completed_batches': []}
    
    def load_pkl_with_cuda_override(self, file_path):
        """Carga PKL sobrescribiendo completamente las referencias CUDA"""
        
        try:
            # Método 1: torch.load con map_location específico
            self.logger.info(f"Método 1: torch.load directo para {file_path.name}")
            
            with open(file_path, 'rb') as f:
                # Usar torch.device explícito
                data = torch.load(f, map_location=torch.device('cpu'))
            
            self.logger.info(f"✓ Método 1 exitoso para {file_path.name}")
            return data
            
        except Exception as e1:
            self.logger.warning(f"Método 1 falló: {str(e1)[:100]}")
            
            try:
                # Método 2: Reemplazar funciones CUDA en tiempo real
                self.logger.info(f"Método 2: Override CUDA para {file_path.name}")
                
                # Backup de funciones originales
                original_cuda_device = torch.cuda.device
                original_cuda_set_device = torch.cuda.set_device
                
                # Sobrescribir funciones CUDA
                torch.cuda.device = lambda x: None
                torch.cuda.set_device = lambda x: None
                
                try:
                    with open(file_path, 'rb') as f:
                        data = torch.load(f, map_location='cpu')
                    
                    self.logger.info(f"✓ Método 2 exitoso para {file_path.name}")
                    return data
                    
                finally:
                    # Restaurar funciones
                    torch.cuda.device = original_cuda_device
                    torch.cuda.set_device = original_cuda_set_device
                    
            except Exception as e2:
                self.logger.warning(f"Método 2 falló: {str(e2)[:100]}")
                
                try:
                    # Método 3: Unpickler con override completo
                    self.logger.info(f"Método 3: Unpickler personalizado para {file_path.name}")
                    
                    class ForceCPUUnpickler(pickle.Unpickler):
                        def find_class(self, module, name):
                            # Interceptar cualquier cosa relacionada con CUDA
                            if 'cuda' in module.lower() or 'cuda' in name.lower():
                                # Redirigir a CPU
                                if name == 'FloatTensor':
                                    return torch.FloatTensor
                                elif name == 'LongTensor':
                                    return torch.LongTensor
                                elif name == 'device':
                                    return lambda x: torch.device('cpu')
                                else:
                                    return super().find_class('torch', name)
                            
                            # Para storage de torch
                            if module == 'torch.storage' and name == '_load_from_bytes':
                                def load_tensor(b):
                                    return torch.load(io.BytesIO(b), map_location='cpu')
                                return load_tensor
                            
                            return super().find_class(module, name)
                    
                    with open(file_path, 'rb') as f:
                        unpickler = ForceCPUUnpickler(f)
                        data = unpickler.load()
                    
                    self.logger.info(f"✓ Método 3 exitoso para {file_path.name}")
                    return data
                    
                except Exception as e3:
                    self.logger.warning(f"Método 3 falló: {str(e3)[:100]}")
                    
                    # Método 4: Extraer solo DataFrames usando búsqueda de patrones
                    return self.extract_dataframes_only(file_path)
    
    def extract_dataframes_only(self, file_path):
        """Método de último recurso: extraer solo DataFrames ignorando modelos"""
        
        self.logger.info(f"Método 4: Extracción de DataFrames para {file_path.name}")
        
        try:
            # Leer archivo completo en memoria
            with open(file_path, 'rb') as f:
                raw_data = f.read()
            
            # Buscar patrones de pandas DataFrame en los bytes
            # Los DataFrames tienen firmas específicas en pickle
            
            # Crear estructura mínima basada en el nombre del archivo
            batch_id = file_path.name.replace("_results.pkl", "")
            parts = batch_id.split("_")
            sentiment = parts[0]
            
            self.logger.warning(f"Creando estructura mínima para {batch_id}")
            
            # Intentar cargar solo metadatos básicos si es posible
            try:
                # Buscar strings JSON embebidos
                import re
                json_pattern = rb'\{"batch_id".*?\}'
                json_matches = re.findall(json_pattern, raw_data)
                
                if json_matches:
                    # Intentar decodificar metadata
                    metadata_str = json_matches[0].decode('utf-8')
                    metadata = json.loads(metadata_str)
                else:
                    metadata = {}
                
            except Exception:
                metadata = {}
            
            # Estructura mínima de respaldo
            result = {
                'batch_id': batch_id,
                'sentiment': sentiment,
                'data': pd.DataFrame(),  # DataFrame vacío - será omitido
                'topic_info': pd.DataFrame({'Topic': [-1], 'Count': [0]}),
                'topic_language_analysis': {},
                'language_distribution': {},
                'n_topics': 0,
                **metadata
            }
            
            self.logger.warning(f"⚠ Estructura mínima creada para {batch_id}")
            return result
            
        except Exception as e:
            self.logger.error(f"Método 4 falló para {file_path.name}: {e}")
            return None
    
    def combine_with_override(self):
        """Combina todos los archivos usando el cargador con override"""
        
        self.logger.info("=== INICIANDO COMBINACIÓN CON OVERRIDE CUDA ===")
        
        result_files = list(self.output_dir.glob("*_results.pkl"))
        self.logger.info(f"Total archivos encontrados: {len(result_files)}")
        
        combined_data = []
        language_analysis = {}
        topic_mappings = {}
        
        successful = 0
        failed = 0
        skipped_empty = 0
        
        for i, file_path in enumerate(result_files):
            batch_id = file_path.name.replace("_results.pkl", "")
            
            self.logger.info(f"\n--- Procesando {i+1}/{len(result_files)}: {batch_id} ---")
            
            if batch_id in self.status.get('completed_batches', []):
                
                # Cargar con override CUDA
                batch_data = self.load_pkl_with_cuda_override(file_path)
                
                if batch_data is not None:
                    # Verificar que tenga datos válidos
                    if ('data' in batch_data and 
                        isinstance(batch_data['data'], pd.DataFrame) and 
                        len(batch_data['data']) > 0):
                        
                        # Procesar datos válidos
                        self._process_valid_batch(
                            batch_data, combined_data, language_analysis, topic_mappings
                        )
                        successful += 1
                        self.logger.info(f"✓ {batch_id}: {len(batch_data['data'])} documentos")
                        
                    else:
                        self.logger.warning(f"⚠ {batch_id}: DataFrame vacío, omitiendo")
                        skipped_empty += 1
                        
                else:
                    self.logger.error(f"✗ {batch_id}: No se pudo cargar")
                    failed += 1
            else:
                self.logger.info(f"- {batch_id}: No en completed_batches")
        
        # Resumen de carga
        self.logger.info(f"\n=== RESUMEN DE CARGA ===")
        self.logger.info(f"Exitosos: {successful}")
        self.logger.info(f"Vacíos omitidos: {skipped_empty}")
        self.logger.info(f"Fallidos: {failed}")
        
        if combined_data:
            self.logger.info("Creando DataFrame final...")
            final_df = pd.concat(combined_data, ignore_index=True)
            
            # Guardar resultado
            output_path = self.output_dir / "combined_multilingual_results_final.pkl"
            final_df.to_pickle(output_path)
            
            # Guardar análisis
            with open(self.output_dir / "multilingual_language_analysis_final.json", 'w') as f:
                json.dump(language_analysis, f, indent=2, default=str)
            
            # Crear resumen final
            summary = {
                'total_documents': len(final_df),
                'successful_batches': successful,
                'skipped_empty': skipped_empty,
                'failed_batches': failed,
                'sentiments': final_df['sentimiento'].value_counts().to_dict() if 'sentimiento' in final_df.columns else {},
                'languages': final_df['idioma'].value_counts().to_dict() if 'idioma' in final_df.columns else {},
                'total_topics': final_df['global_topic'].nunique() if 'global_topic' in final_df.columns else 0
            }
            
            with open(self.output_dir / "final_summary_override.json", 'w') as f:
                json.dump(summary, f, indent=2)
            
            self.logger.info(f"🎉 ÉXITO: {len(final_df):,} documentos combinados")
            return final_df, {}, language_analysis, summary
            
        else:
            self.logger.error("💥 FALLO: No se pudieron combinar datos")
            return None, None, None, None
    
    def _process_valid_batch(self, batch_data, combined_data, language_analysis, topic_mappings):
        """Procesa un lote con datos válidos"""
        
        sentiment = batch_data.get('sentiment', 'unknown')
        df = batch_data['data']
        
        # Inicializar estructuras
        if sentiment not in topic_mappings:
            topic_mappings[sentiment] = {}
            language_analysis[sentiment] = {}
        
        # Remapear temas
        if 'topic' in df.columns:
            n_existing = len([t for t in topic_mappings[sentiment].values() if t != -1])
            
            for topic in df['topic'].unique():
                if topic not in topic_mappings[sentiment]:
                    topic_mappings[sentiment][topic] = -1 if topic == -1 else n_existing
                    if topic != -1:
                        n_existing += 1
            
            df['global_topic'] = df['topic'].map(topic_mappings[sentiment])
        
        # Agregar metadatos
        df['batch_lang_group'] = batch_data.get('lang_group', 'unknown')
        
        combined_data.append(df)
        
        # Procesar análisis de idiomas
        topic_lang = batch_data.get('topic_language_analysis', {})
        for topic_id, langs in topic_lang.items():
            mapped_topic = topic_mappings[sentiment].get(topic_id, topic_id)
            if mapped_topic not in language_analysis[sentiment]:
                language_analysis[sentiment][mapped_topic] = {}
            
            for lang, count in langs.items():
                if lang not in language_analysis[sentiment][mapped_topic]:
                    language_analysis[sentiment][mapped_topic][lang] = 0
                language_analysis[sentiment][mapped_topic][lang] += count


def ultimate_combine_results(output_dir="E:/bertopic_800k_multilingue"):
    """FUNCIÓN DEFINITIVA - Combina archivos con override CUDA completo"""
    
    print("=== COMBINACIÓN DEFINITIVA CON OVERRIDE CUDA ===")
    print(f"Directorio: {output_dir}")
    
    loader = DefinitiveCudaLoader(output_dir)
    result = loader.combine_with_override()
    
    if result[0] is not None:
        final_df, models, lang_analysis, summary = result
        
        print(f"\n🎊 ÉXITO DEFINITIVO 🎊")
        print(f"Total documentos: {len(final_df):,}")
        print(f"Lotes procesados exitosamente: {summary.get('successful_batches', 0)}")
        print(f"Lotes omitidos (vacíos): {summary.get('skipped_empty', 0)}")
        print(f"Lotes fallidos: {summary.get('failed_batches', 0)}")
        
        if 'sentimiento' in final_df.columns:
            print(f"Sentimientos: {list(final_df['sentimiento'].unique())}")
        if 'idioma' in final_df.columns:
            print(f"Idiomas: {final_df['idioma'].nunique()}")
        if 'global_topic' in final_df.columns:
            print(f"Temas: {final_df['global_topic'].nunique()}")
        
        return final_df, models, lang_analysis, summary
    else:
        print("❌ Falló la combinación definitiva")
        return None, None, None, None

## ⚙️ Configuración de BERTopic
- Define hiperparámetros/pipe (vectorizador, umap/hdbscan, idioma).
- Llama a la función ultimate_combine_results con el directorio "E:/bertopic_800k_multilingue" y desempaqueta lo que devuelve en cuatro variables.

In [ ]:
final_df, models, lang_analysis, summary = ultimate_combine_results("E:/bertopic_800k_multilingue")

=== COMBINACIÓN DEFINITIVA CON OVERRIDE CUDA ===
Directorio: E:/bertopic_800k_multilingue


INFO:__main__:=== INICIANDO COMBINACIÓN CON OVERRIDE CUDA ===
INFO:__main__:Total archivos encontrados: 216
INFO:__main__:
--- Procesando 1/216: negativo_main_batch_000 ---
INFO:__main__:Método 1: torch.load directo para negativo_main_batch_000_results.pkl
C:\Users\swart\AppData\Local\Temp\ipykernel_16128\86875392.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend

## 🧰 Librerías e importaciones
- Prepara todas las librerías necesarias para cargar/combinar pickles y trabajar con datos y registros en CPU.

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import torch
import pickle
import pandas as pd
from pathlib import Path
import json
import logging
import io

## 📥 Carga de datos
- Lee el dataset de entrada (processing_status.json, multilingual_language_analysis_method3.json).

In [3]:
class OptimizedMethod3Combiner:
    """Combinador optimizado que usa solo el método 3 (más confiable)"""
    
    def __init__(self, output_dir):
        self.output_dir = Path(output_dir)
        self.status_file = self.output_dir / 'processing_status.json'
        
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
        self.load_processing_status()
    
    def load_processing_status(self):
        if self.status_file.exists():
            with open(self.status_file, 'r') as f:
                self.status = json.load(f)
        else:
            self.status = {'completed_batches': []}
    
    def load_with_cpu_unpickler(self, file_path):
        """Carga usando solo el método 3 optimizado"""
        
        class OptimizedCPUUnpickler(pickle.Unpickler):
            def find_class(self, module, name):
                # Interceptar cualquier cosa relacionada con CUDA
                if 'cuda' in module.lower() or 'cuda' in name.lower():
                    if name == 'FloatTensor':
                        return torch.FloatTensor
                    elif name == 'LongTensor':
                        return torch.LongTensor
                    elif name == 'device':
                        return lambda x: torch.device('cpu')
                    else:
                        return super().find_class('torch', name)
                
                # Para storage de torch
                if module == 'torch.storage' and name == '_load_from_bytes':
                    def load_tensor(b):
                        return torch.load(io.BytesIO(b), map_location='cpu')
                    return load_tensor
                
                return super().find_class(module, name)
        
        try:
            with open(file_path, 'rb') as f:
                unpickler = OptimizedCPUUnpickler(f)
                data = unpickler.load()
            
            self.logger.info(f"✓ Cargado exitosamente: {file_path.name}")
            return data
            
        except Exception as e:
            self.logger.error(f"✗ Error cargando {file_path.name}: {str(e)[:100]}")
            return None
    
    def combine_results_method3_only(self):
        """Combina resultados usando solo método 3"""
        
        self.logger.info("=== COMBINACIÓN MÉTODO 3 OPTIMIZADO ===")
        
        result_files = list(self.output_dir.glob("*_results.pkl"))
        self.logger.info(f"Archivos encontrados: {len(result_files)}")
        
        combined_data = []
        language_analysis = {}
        topic_mappings = {}
        
        successful = 0
        failed = 0
        empty_skipped = 0
        
        for i, file_path in enumerate(result_files):
            batch_id = file_path.name.replace("_results.pkl", "")
            
            # Progreso cada 10 archivos
            if (i + 1) % 10 == 0 or i == 0:
                self.logger.info(f"Progreso: {i+1}/{len(result_files)} ({(i+1)/len(result_files)*100:.1f}%)")
            
            if batch_id in self.status.get('completed_batches', []):
                # Cargar con método 3
                batch_data = self.load_with_cpu_unpickler(file_path)
                
                if batch_data is not None:
                    if (isinstance(batch_data, dict) and 
                        'data' in batch_data and 
                        isinstance(batch_data['data'], pd.DataFrame) and 
                        len(batch_data['data']) > 0):
                        
                        # Procesar datos válidos
                        self._process_batch_data(
                            batch_data, combined_data, language_analysis, topic_mappings
                        )
                        successful += 1
                        
                    else:
                        self.logger.debug(f"DataFrame vacío en {batch_id}")
                        empty_skipped += 1
                else:
                    failed += 1
        
        # Resumen final
        self.logger.info(f"\n=== RESULTADOS FINALES ===")
        self.logger.info(f"Exitosos: {successful}")
        self.logger.info(f"Vacíos omitidos: {empty_skipped}")
        self.logger.info(f"Fallidos: {failed}")
        self.logger.info(f"Total válidos: {successful}/{len(result_files)} ({successful/len(result_files)*100:.1f}%)")
        
        if combined_data:
            self.logger.info("Creando DataFrame final...")
            final_df = pd.concat(combined_data, ignore_index=True)
            
            # Guardar resultado
            output_path = self.output_dir / "combined_multilingual_results_method3.pkl"
            final_df.to_pickle(output_path)
            
            # Guardar análisis de idiomas
            lang_file = self.output_dir / "multilingual_language_analysis_method3.json"
            with open(lang_file, 'w') as f:
                json.dump(language_analysis, f, indent=2, default=str)
            
            # Crear resumen completo
            summary = self.create_comprehensive_summary(final_df, language_analysis)
            
            self.logger.info(f"COMBINACIÓN COMPLETADA: {len(final_df):,} documentos")
            self.logger.info(f"Archivo guardado: {output_path}")
            
            return final_df, {}, language_analysis, summary
            
        else:
            self.logger.error("No se pudieron combinar datos")
            return None, None, None, None
    
    def _process_batch_data(self, batch_data, combined_data, language_analysis, topic_mappings):
        """Procesa datos de un lote válido"""
        
        sentiment = batch_data.get('sentiment', 'unknown')
        df = batch_data['data'].copy()
        
        # Inicializar estructuras para este sentimiento
        if sentiment not in topic_mappings:
            topic_mappings[sentiment] = {}
            language_analysis[sentiment] = {}
        
        # Remapear temas para evitar conflictos
        if 'topic' in df.columns:
            n_existing_topics = len([t for t in topic_mappings[sentiment].values() if t != -1])
            
            for topic in df['topic'].unique():
                if topic not in topic_mappings[sentiment]:
                    if topic == -1:
                        topic_mappings[sentiment][topic] = -1
                    else:
                        topic_mappings[sentiment][topic] = n_existing_topics
                        n_existing_topics += 1
            
            # Aplicar mapeo global
            df['global_topic'] = df['topic'].map(topic_mappings[sentiment])
        
        # Agregar metadatos del lote
        df['batch_lang_group'] = batch_data.get('lang_group', 'unknown')
        df['batch_id'] = batch_data.get('batch_id', 'unknown')
        
        combined_data.append(df)
        
        # Procesar análisis de idiomas por tema
        topic_lang = batch_data.get('topic_language_analysis', {})
        for topic_id, langs in topic_lang.items():
            mapped_topic = topic_mappings[sentiment].get(topic_id, topic_id)
            
            if mapped_topic not in language_analysis[sentiment]:
                language_analysis[sentiment][mapped_topic] = {}
            
            for lang, count in langs.items():
                if lang not in language_analysis[sentiment][mapped_topic]:
                    language_analysis[sentiment][mapped_topic][lang] = 0
                language_analysis[sentiment][mapped_topic][lang] += count
    
    def create_comprehensive_summary(self, df, language_analysis):
        """Crea resumen completo de los datos combinados"""
        
        summary = {
            'processing_info': {
                'total_documents': len(df),
                'processing_date': pd.Timestamp.now().isoformat(),
                'unique_batches': df['batch_id'].nunique() if 'batch_id' in df.columns else 0
            },
            'content_analysis': {
                'unique_sentiments': df['sentimiento'].nunique() if 'sentimiento' in df.columns else 0,
                'unique_languages': df['idioma'].nunique() if 'idioma' in df.columns else 0,
                'unique_topics': df['global_topic'].nunique() if 'global_topic' in df.columns else 0,
                'outliers_count': len(df[df['global_topic'] == -1]) if 'global_topic' in df.columns else 0
            },
            'distributions': {
                'by_sentiment': df['sentimiento'].value_counts().to_dict() if 'sentimiento' in df.columns else {},
                'by_language': df['idioma'].value_counts().to_dict() if 'idioma' in df.columns else {},
                'by_batch_group': df['batch_lang_group'].value_counts().to_dict() if 'batch_lang_group' in df.columns else {}
            },
            'topics_analysis': {}
        }
        
        # Análisis de temas por sentimiento
        if 'sentimiento' in df.columns and 'global_topic' in df.columns:
            for sentiment in df['sentimiento'].unique():
                sent_data = df[df['sentimiento'] == sentiment]
                n_topics = sent_data['global_topic'].nunique()
                if -1 in sent_data['global_topic'].values:
                    n_topics -= 1
                
                summary['topics_analysis'][sentiment] = {
                    'total_docs': len(sent_data),
                    'total_topics': n_topics,
                    'outliers': len(sent_data[sent_data['global_topic'] == -1]),
                    'avg_docs_per_topic': len(sent_data) / n_topics if n_topics > 0 else 0
                }
        
        # Top idiomas y sentimientos
        if 'idioma' in df.columns:
            top_languages = df['idioma'].value_counts().head(10)
            summary['top_languages'] = top_languages.to_dict()
        
        # Guardar resumen
        summary_file = self.output_dir / "comprehensive_summary_method3.json"
        with open(summary_file, 'w') as f:
            json.dump(summary, f, indent=2, default=str)
        
        return summary


def optimized_combine_results(output_dir="E:/bertopic_800k_multilingue"):
    """Función optimizada que usa solo método 3"""
    
    print("=== COMBINACIÓN OPTIMIZADA (MÉTODO 3 SOLAMENTE) ===")
    print(f"Directorio: {output_dir}")
    
    combiner = OptimizedMethod3Combiner(output_dir)
    result = combiner.combine_results_method3_only()
    
    if result[0] is not None:
        final_df, models, lang_analysis, summary = result
        
        print(f"\n✓ ÉXITO USANDO MÉTODO 3")
        print(f"Documentos totales: {len(final_df):,}")
        print(f"Lotes exitosos: {summary['processing_info'].get('unique_batches', 'N/A')}")
        
        if 'sentimiento' in final_df.columns:
            sentiments = list(final_df['sentimiento'].unique())
            print(f"Sentimientos: {sentiments}")
            
        if 'idioma' in final_df.columns:
            n_languages = final_df['idioma'].nunique()
            top_lang = final_df['idioma'].value_counts().index[0]
            print(f"Idiomas: {n_languages} (principal: {top_lang})")
            
        if 'global_topic' in final_df.columns:
            n_topics = final_df['global_topic'].nunique()
            if -1 in final_df['global_topic'].values:
                n_topics -= 1
            print(f"Temas identificados: {n_topics}")
        
        return final_df, models, lang_analysis, summary
    else:
        print("✗ Falló la combinación")
        return None, None, None, None

## ⚙️ Configuración de BERTopic
- Define hiperparámetros/pipe (vectorizador, umap/hdbscan, idioma).

In [ ]:
final_df, models, lang_analysis, summary = optimized_combine_results("E:/bertopic_800k_multilingue")

=== COMBINACIÓN OPTIMIZADA (MÉTODO 3 SOLAMENTE) ===
Directorio: E:/bertopic_800k_multilingue


INFO:__main__:=== COMBINACIÓN MÉTODO 3 OPTIMIZADO ===
INFO:__main__:Archivos encontrados: 216
INFO:__main__:Progreso: 1/216 (0.5%)
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_000_results.pkl
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_001_results.pkl
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_002_results.pkl
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_003_results.pkl
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_004_results.pkl
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_005_results.pkl
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_006_results.pkl
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_007_results.pkl
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_008_results.pkl
INFO:__main__:Progreso: 10/216 (4.6%)
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_009_results.pkl
INFO:__main__:✓ Cargado exitosamente: negativo_main_batch_010_results.pkl
INFO:__main__:✓ C

## 📦 Dependencias
- Importa pandas, numpy para el flujo de tópicos.
- Resiliente a pickles con CUDA (fuerza CPU).
- Baja memoria (lotes + parciales comprimidos).
- Reanuda sin repetir trabajo (estado JSON-safe).

In [14]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import torch
import pickle
import pandas as pd
from pathlib import Path
import json
import logging
import io
import gc

class IncrementalMemoryManagedCombiner:
    """Combinador que procesa en lotes pequeños y libera memoria constantemente"""
    
    def __init__(self, output_dir, batch_size=10, auto_save_every=5):
        self.output_dir = Path(output_dir)
        self.status_file = self.output_dir / 'processing_status.json'
        self.batch_size = batch_size  # Cuántos archivos procesar antes de guardar
        self.auto_save_every = auto_save_every  # Cada cuántos lotes guardar progreso
        
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
        self.load_processing_status()
        
        # Archivos de progreso incremental
        self.progress_dir = self.output_dir / "incremental_progress"
        self.progress_dir.mkdir(exist_ok=True)
        
        # Estado de combinación
        self.combination_state = self.load_combination_state()
    def convert_to_json_safe(self, obj):
        """Convierte tipos numpy/pandas a tipos JSON-serializables"""
        import numpy as np
        
        if isinstance(obj, dict):
            return {str(k): self.convert_to_json_safe(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [self.convert_to_json_safe(item) for item in obj]
        elif isinstance(obj, (np.integer, np.int64, np.int32)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj
    def load_processing_status(self):
        if self.status_file.exists():
            with open(self.status_file, 'r') as f:
                self.status = json.load(f)
        else:
            self.status = {'completed_batches': []}
    
    def load_combination_state(self):
        """Carga el estado de la combinación incremental"""
        state_file = self.progress_dir / "combination_state.json"
        
        if state_file.exists():
            with open(state_file, 'r') as f:
                state = json.load(f)
            self.logger.info(f"Recuperando desde lote procesado: {state.get('last_processed', 0)}")
            return state
        else:
            return {
                'last_processed': 0,
                'total_documents': 0,
                'processed_files': [],
                'topic_mappings': {},
                'language_analysis': {},
                'sentiment_counts': {},
                'partial_results_files': []
            }
    
    def save_combination_state(self):
        """Guarda el estado actual de la combinación (versión JSON-safe)"""
        state_file = self.progress_dir / "combination_state.json"
        
        # Convertir estado a JSON-safe
        safe_state = self.convert_to_json_safe(self.combination_state)
        
        with open(state_file, 'w') as f:
            json.dump(safe_state, f, indent=2)
    
    def load_with_cpu_unpickler(self, file_path):
        """Método 3 optimizado con limpieza de memoria"""
        
        class MemoryEfficientUnpickler(pickle.Unpickler):
            def find_class(self, module, name):
                if 'cuda' in module.lower() or 'cuda' in name.lower():
                    if name == 'FloatTensor':
                        return torch.FloatTensor
                    elif name == 'LongTensor':
                        return torch.LongTensor
                    elif name == 'device':
                        return lambda x: torch.device('cpu')
                    else:
                        return super().find_class('torch', name)
                
                if module == 'torch.storage' and name == '_load_from_bytes':
                    def load_tensor(b):
                        tensor = torch.load(io.BytesIO(b), map_location='cpu')
                        return tensor
                    return load_tensor
                
                return super().find_class(module, name)
        
        try:
            with open(file_path, 'rb') as f:
                unpickler = MemoryEfficientUnpickler(f)
                data = unpickler.load()
            
            return data
            
        except Exception as e:
            self.logger.error(f"Error cargando {file_path.name}: {str(e)[:100]}")
            return None
    
    def process_batch_of_files(self, file_batch):
        """Procesa un lote pequeño de archivos y extrae solo lo esencial"""
        
        batch_dataframes = []
        
        for file_path in file_batch:
            batch_id = file_path.name.replace("_results.pkl", "")
            
            if batch_id in self.status.get('completed_batches', []):
                # Cargar archivo
                self.logger.info(f"Cargando {batch_id}...")
                batch_data = self.load_with_cpu_unpickler(file_path)
                
                if (batch_data is not None and 
                    isinstance(batch_data, dict) and 
                    'data' in batch_data and 
                    isinstance(batch_data['data'], pd.DataFrame) and 
                    len(batch_data['data']) > 0):
                    
                    # Extraer solo DataFrame esencial
                    df = batch_data['data'].copy()
                    sentiment = batch_data.get('sentiment', 'unknown')
                    
                    # Procesar mapeo de temas
                    self._update_topic_mappings(df, sentiment)
                    
                    # Procesar análisis de idiomas
                    self._update_language_analysis(batch_data, sentiment)
                    
                    # Agregar metadatos mínimos
                    df['sentiment_processed'] = sentiment
                    df['batch_id_processed'] = batch_id
                    
                    batch_dataframes.append(df)
                    
                    # Actualizar contadores
                    self.combination_state['total_documents'] += len(df)
                    if sentiment not in self.combination_state['sentiment_counts']:
                        self.combination_state['sentiment_counts'][sentiment] = 0
                    self.combination_state['sentiment_counts'][sentiment] += len(df)
                    
                    self.logger.info(f"Procesado {batch_id}: {len(df)} docs")
                
                # CRÍTICO: Liberar memoria del batch_data inmediatamente
                del batch_data
                gc.collect()
        
        return batch_dataframes
    
    def _update_topic_mappings(self, df, sentiment):
        """Actualiza mapeos de temas incrementalmente (versión JSON-safe)"""
        
        if sentiment not in self.combination_state['topic_mappings']:
            self.combination_state['topic_mappings'][sentiment] = {}
        
        if 'topic' in df.columns:
            mappings = self.combination_state['topic_mappings'][sentiment]
            n_existing = len([t for t in mappings.values() if t != -1])
            
            for topic in df['topic'].unique():
                # Convertir topic a int estándar de Python
                topic_key = int(topic) if hasattr(topic, 'dtype') else topic
                
                if str(topic_key) not in mappings:  # Usar string como clave
                    if topic_key == -1:
                        mappings[str(topic_key)] = -1
                    else:
                        mappings[str(topic_key)] = n_existing
                        n_existing += 1
            
            # Aplicar mapeo (convirtiendo claves de vuelta a int para pandas)
            topic_map = {int(k): v for k, v in mappings.items()}
            df['global_topic'] = df['topic'].map(topic_map)
    
    def _update_language_analysis(self, batch_data, sentiment):
        """Actualiza análisis de idiomas incrementalmente (versión JSON-safe)"""
        
        if sentiment not in self.combination_state['language_analysis']:
            self.combination_state['language_analysis'][sentiment] = {}
        
        topic_lang = batch_data.get('topic_language_analysis', {})
        sentiment_analysis = self.combination_state['language_analysis'][sentiment]
        
        for topic_id, langs in topic_lang.items():
            # Convertir topic_id a string para JSON
            topic_key = str(int(topic_id)) if hasattr(topic_id, 'dtype') else str(topic_id)
            mapped_topic_id = self.combination_state['topic_mappings'][sentiment].get(topic_key, topic_id)
            mapped_topic_key = str(mapped_topic_id)
            
            if mapped_topic_key not in sentiment_analysis:
                sentiment_analysis[mapped_topic_key] = {}
            
            for lang, count in langs.items():
                if lang not in sentiment_analysis[mapped_topic_key]:
                    sentiment_analysis[mapped_topic_key][lang] = 0
                # Convertir count a int estándar
                count_safe = int(count) if hasattr(count, 'dtype') else count
                sentiment_analysis[mapped_topic_key][lang] += count_safe
    
    def save_partial_results(self, combined_df, part_number):
        """Guarda resultados parciales y libera memoria"""
        
        partial_file = self.progress_dir / f"partial_results_{part_number:03d}.pkl"
        
        # Guardar DataFrame parcial
        combined_df.to_pickle(partial_file, compression='gzip')
        
        # Registrar archivo parcial
        self.combination_state['partial_results_files'].append(str(partial_file))
        
        self.logger.info(f"Guardado parcial {part_number}: {len(combined_df)} docs en {partial_file.name}")
        
        # Liberar memoria
        del combined_df
        gc.collect()
        
        return partial_file
    
    def combine_incremental_with_memory_management(self):
        """Combinador principal con gestión de memoria"""
        
        self.logger.info("=== COMBINACIÓN INCREMENTAL CON GESTIÓN DE MEMORIA ===")
        
        # Obtener archivos a procesar
        all_result_files = list(self.output_dir.glob("*_results.pkl"))
        completed_files = [f for f in all_result_files 
                          if f.name.replace("_results.pkl", "") in self.status.get('completed_batches', [])]
        
        self.logger.info(f"Total archivos a procesar: {len(completed_files)}")
        
        # Filtrar archivos ya procesados
        processed_names = [Path(f).name for f in self.combination_state.get('processed_files', [])]
        remaining_files = [f for f in completed_files if f.name not in processed_names]
        
        self.logger.info(f"Archivos pendientes: {len(remaining_files)}")
        
        if not remaining_files:
            self.logger.info("Todos los archivos ya procesados, combinando resultados finales...")
            return self.combine_partial_results()
        
        # Procesar en lotes pequeños
        part_number = len(self.combination_state.get('partial_results_files', []))
        
        for i in range(0, len(remaining_files), self.batch_size):
            batch_files = remaining_files[i:i+self.batch_size]
            
            self.logger.info(f"Procesando lote {i//self.batch_size + 1}: archivos {i+1}-{min(i+self.batch_size, len(remaining_files))}")
            
            # Procesar lote de archivos
            batch_dataframes = self.process_batch_of_files(batch_files)
            
            if batch_dataframes:
                # Combinar DataFrames del lote
                combined_batch = pd.concat(batch_dataframes, ignore_index=True)
                
                # Guardar resultado parcial
                part_number += 1
                self.save_partial_results(combined_batch, part_number)
                
                # Actualizar estado
                self.combination_state['last_processed'] = i + len(batch_files)
                self.combination_state['processed_files'].extend([str(f) for f in batch_files])
                
                # Guardar estado cada auto_save_every lotes
                if part_number % self.auto_save_every == 0:
                    self.save_combination_state()
                    self.logger.info(f"Estado guardado - Progreso: {self.combination_state['last_processed']}/{len(remaining_files)}")
            
            # Limpiar memoria después de cada lote
            if 'batch_dataframes' in locals():
                del batch_dataframes
            if 'combined_batch' in locals():
                del combined_batch
            gc.collect()
            
            self.logger.info(f"Memoria liberada después del lote {part_number}")
        
        # Guardar estado final
        self.save_combination_state()
        
        # Combinar todos los resultados parciales
        self.logger.info("Combinando todos los resultados parciales...")
        return self.combine_partial_results()
    
    def combine_partial_results(self):
        """Combina todos los archivos parciales en resultado final"""
        
        partial_files = self.combination_state.get('partial_results_files', [])
        
        if not partial_files:
            self.logger.error("No hay archivos parciales para combinar")
            return None, None, None, None
        
        self.logger.info(f"Combinando {len(partial_files)} archivos parciales...")
        
        # Combinar archivos parciales uno por uno
        final_dataframes = []
        
        for partial_file in partial_files:
            self.logger.info(f"Cargando {Path(partial_file).name}...")
            
            try:
                partial_df = pd.read_pickle(partial_file, compression='gzip')
                final_dataframes.append(partial_df)
                
                # Liberar inmediatamente
                del partial_df
                gc.collect()
                
            except Exception as e:
                self.logger.error(f"Error cargando {partial_file}: {e}")
                continue
        
        if final_dataframes:
            self.logger.info("Creando DataFrame final...")
            final_df = pd.concat(final_dataframes, ignore_index=True)
            
            # Guardar resultado final
            output_path = self.output_dir / "combined_multilingual_results_incremental.pkl"
            final_df.to_pickle(output_path, compression='gzip')
            
            # Crear resumen final
            summary = {
                'total_documents': len(final_df),
                'total_partial_files': len(partial_files),
                'sentiment_counts': self.combination_state['sentiment_counts'],
                'topic_mappings_summary': {k: len(v) for k, v in self.combination_state['topic_mappings'].items()},
                'processing_completed': True
            }
            
            with open(self.output_dir / "incremental_summary.json", 'w') as f:
                json.dump(summary, f, indent=2)
            
            self.logger.info(f"COMBINACIÓN INCREMENTAL COMPLETADA: {len(final_df):,} documentos")
            
            return final_df, {}, self.combination_state['language_analysis'], summary
        
        else:
            self.logger.error("No se pudieron cargar archivos parciales")
            return None, None, None, None


def incremental_combine_results(output_dir="E:/bertopic_800k_multilingue", batch_size=8):
    """Función principal para combinación incremental"""
    
    print(f"=== COMBINACIÓN INCREMENTAL (LOTES DE {batch_size}) ===")
    print(f"Directorio: {output_dir}")
    
    combiner = IncrementalMemoryManagedCombiner(
        output_dir=output_dir, 
        batch_size=batch_size,  # Procesar solo 8 archivos a la vez
        auto_save_every=3       # Guardar estado cada 3 lotes
    )
    
    result = combiner.combine_incremental_with_memory_management()
    
    if result[0] is not None:
        final_df, models, lang_analysis, summary = result
        
        print(f"\nCOMBINACIÓN INCREMENTAL EXITOSA")
        print(f"Total documentos: {len(final_df):,}")
        print(f"Archivos parciales procesados: {summary.get('total_partial_files', 0)}")
        print(f"Distribución por sentimiento: {summary.get('sentiment_counts', {})}")
        
        return final_df, models, lang_analysis, summary
    else:
        print("Falló la combinación incremental")
        return None, None, None, None

## ⚙️ Configuración de BERTopic
- Define hiperparámetros/pipe (vectorizador, umap/hdbscan, idioma).

In [18]:
final_df, models, lang_analysis, summary = incremental_combine_results("E:/bertopic_800k_multilingue", batch_size=10)

INFO:__main__:=== COMBINACIÓN INCREMENTAL CON GESTIÓN DE MEMORIA ===
INFO:__main__:Total archivos a procesar: 216
INFO:__main__:Archivos pendientes: 216
INFO:__main__:Procesando lote 1: archivos 1-10
INFO:__main__:Cargando negativo_main_batch_000...


=== COMBINACIÓN INCREMENTAL (LOTES DE 10) ===
Directorio: E:/bertopic_800k_multilingue


INFO:__main__:Procesado negativo_main_batch_000: 4000 docs
INFO:__main__:Cargando negativo_main_batch_001...
INFO:__main__:Procesado negativo_main_batch_001: 4000 docs
INFO:__main__:Cargando negativo_main_batch_002...
INFO:__main__:Procesado negativo_main_batch_002: 4000 docs
INFO:__main__:Cargando negativo_main_batch_003...
INFO:__main__:Procesado negativo_main_batch_003: 4000 docs
INFO:__main__:Cargando negativo_main_batch_004...
INFO:__main__:Procesado negativo_main_batch_004: 4000 docs
INFO:__main__:Cargando negativo_main_batch_005...
INFO:__main__:Procesado negativo_main_batch_005: 4000 docs
INFO:__main__:Cargando negativo_main_batch_006...
INFO:__main__:Procesado negativo_main_batch_006: 4000 docs
INFO:__main__:Cargando negativo_main_batch_007...
INFO:__main__:Procesado negativo_main_batch_007: 4000 docs
INFO:__main__:Cargando negativo_main_batch_008...
INFO:__main__:Procesado negativo_main_batch_008: 4000 docs
INFO:__main__:Cargando negativo_main_batch_009...
INFO:__main__:Proce


COMBINACIÓN INCREMENTAL EXITOSA
Total documentos: 852,566
Archivos parciales procesados: 22
Distribución por sentimiento: {'negativo': 101666, 'neutro': 424614, 'positivo': 326286}


- Ejecución auxiliar dentro del flujo.

In [26]:
final_df.columns

Index(['texto', 'ciudad', 'categoria', 'fecha', 'fuente', 'sentimiento',
       'confianza', 'descripcion_sencilla', 'idioma', 'mes', 'año', 'topic',
       'batch_id', 'global_topic', 'sentiment_processed',
       'batch_id_processed'],
      dtype='object')

## 🔍 Visualización de Data Frame
- Ejecución auxiliar dentro del flujo.

In [44]:
final_df.head()

,texto,ciudad,categoria,fecha,fuente,sentimiento,confianza,descripcion_sencilla,idioma,mes,año,topic,batch_id,global_topic,sentiment_processed,batch_id_processed
0,Nous sommes arrivés en retard et le monsieur à...,Barcelona,Playa,2025-07-17,Booking,negativo,0.998544,"Día cielo nublado, cálido, sin lluvia",fr,7.0,2025.0,1,negativo_main_batch_000,0,negativo,negativo_main_batch_000
1,Read very carefully what you are booking. You ...,Barcelona,Tour,2025-07-11,Booking,negativo,0.999205,"Día con lluvia moderada, cálido",en,7.0,2025.0,1,negativo_main_batch_000,0,negativo,negativo_main_batch_000
2,We came during the last hour of admission so w...,Barcelona,Tour,2025-07-08,Booking,negativo,0.996447,"Día cielo con intervalos, cálido, sin lluvia",en,7.0,2025.0,1,negativo_main_batch_000,0,negativo,negativo_main_batch_000
3,we booked disabled access but needed proof of ...,Barcelona,Tour,2025-07-06,Booking,negativo,0.999162,"Día cielo con intervalos, cálido, sin lluvia",en,7.0,2025.0,1,negativo_main_batch_000,0,negativo,negativo_main_batch_000
4,Booking no nos dio el código para descargar la...,Barcelona,Tour,2025-07-03,Booking,negativo,0.999408,"Día cielo con intervalos, cálido, sin lluvia",es,7.0,2025.0,1,negativo_main_batch_000,0,negativo,negativo_main_batch_000


## ⚙️ Configuración de BERTopic
- Define hiperparámetros/pipe (vectorizador, umap/hdbscan, idioma).

In [28]:
def analyze_multilingual_topics(final_df, lang_analysis, sentiment='positivo', top_n=10):
    """
    📈 FUNCIÓN DE ANÁLISIS: Analiza temas multilingües específicos
    Adaptada para tu DataFrame con columnas específicas
    """
    print(f"=== ANÁLISIS DE TEMAS MULTILINGÜES ({sentiment.upper()}) ===")
    
    # Filtrar por sentimiento usando la columna correcta
    sent_data = final_df[final_df['sentimiento'] == sentiment]
    
    if len(sent_data) == 0:
        print(f"No hay datos para el sentimiento: {sentiment}")
        return
    
    print(f"Total documentos para {sentiment}: {len(sent_data):,}")
    
    # Top temas más frecuentes
    topic_counts = sent_data['global_topic'].value_counts().head(top_n)
    print(f"\nTop {top_n} temas más frecuentes:")
    
    for i, (topic_id, count) in enumerate(topic_counts.items(), 1):
        if topic_id != -1:  # Excluir outliers
            pct = (count / len(sent_data)) * 100
            print(f"\n{i}. Tema {topic_id}: {count:,} documentos ({pct:.1f}%)")
            
            # Filtrar documentos de este tema
            topic_docs = sent_data[sent_data['global_topic'] == topic_id]
            
            # Análisis de idiomas para este tema
            lang_dist = topic_docs['idioma'].value_counts()
            print(f"   Idiomas: {dict(lang_dist.head(5))}")
            
            # Diversidad lingüística
            diversity = len(lang_dist)
            main_lang = lang_dist.index[0] if len(lang_dist) > 0 else 'unknown'
            main_lang_pct = (lang_dist.iloc[0] / len(topic_docs)) * 100 if len(lang_dist) > 0 else 0
            print(f"   Diversidad: {diversity} idiomas, principal: {main_lang} ({main_lang_pct:.1f}%)")
            
            # Análisis temporal si tienes fechas
            if 'fecha' in topic_docs.columns:
                years = topic_docs['año'].value_counts().head(3)
                print(f"   Años principales: {dict(years)}")
            
            # Análisis geográfico si tienes ciudades
            if 'ciudad' in topic_docs.columns:
                cities = topic_docs['ciudad'].value_counts().head(3)
                print(f"   Ciudades principales: {dict(cities)}")
            
            # Análisis de categorías si está disponible
            if 'categoria' in topic_docs.columns:
                categories = topic_docs['categoria'].value_counts().head(3)
                print(f"   Categorías: {dict(categories)}")
            
            # Ejemplos de documentos
            print("   Ejemplos de documentos:")
            sample_docs = topic_docs.sample(min(3, len(topic_docs)))
            
            for j, (_, doc) in enumerate(sample_docs.iterrows(), 1):
                text_preview = doc['texto'][:100] + "..." if len(doc['texto']) > 100 else doc['texto']
                ciudad_info = f" - {doc['ciudad']}" if pd.notna(doc.get('ciudad')) else ""
                fecha_info = f" ({doc['año']})" if pd.notna(doc.get('año')) else ""
                print(f"      {j}. [{doc['idioma']}]{ciudad_info}{fecha_info}: {text_preview}")

def analyze_all_sentiments(final_df, lang_analysis, top_n=10):
    """Analiza todos los sentimientos disponibles"""
    
    print("=== ANÁLISIS COMPLETO DE TODOS LOS SENTIMIENTOS ===")
    
    sentiments = final_df['sentimiento'].unique()
    print(f"Sentimientos encontrados: {list(sentiments)}")
    
    for sentiment in sentiments:
        print("\n" + "="*60)
        analyze_multilingual_topics(final_df, lang_analysis, sentiment=sentiment, top_n=top_n)

def create_topic_summary_report(final_df, lang_analysis):
    """Crea un reporte resumen de todos los temas"""
    
    print("=== REPORTE RESUMEN DE TEMAS ===")
    
    # Estadísticas generales
    total_docs = len(final_df)
    total_topics = final_df['global_topic'].nunique() - (1 if -1 in final_df['global_topic'].values else 0)
    outliers = len(final_df[final_df['global_topic'] == -1])
    
    print(f"Total documentos: {total_docs:,}")
    print(f"Total temas identificados: {total_topics}")
    print(f"Documentos sin clasificar (outliers): {outliers:,} ({outliers/total_docs*100:.1f}%)")
    
    # Por sentimiento
    print(f"\nDistribución por sentimiento:")
    sentiment_stats = final_df.groupby('sentimiento').agg({
        'global_topic': lambda x: x.nunique() - (1 if -1 in x.values else 0),
        'texto': 'count',
        'idioma': 'nunique'
    }).round(2)
    
    sentiment_stats.columns = ['Temas', 'Documentos', 'Idiomas']
    print(sentiment_stats)
    
    # Top idiomas globales
    print(f"\nTop 10 idiomas:")
    lang_dist = final_df['idioma'].value_counts().head(10)
    for lang, count in lang_dist.items():
        pct = (count / total_docs) * 100
        print(f"  {lang}: {count:,} ({pct:.1f}%)")
    
    # Análisis temporal
    if 'año' in final_df.columns:
        print(f"\nDistribución temporal:")
        year_dist = final_df['año'].value_counts().sort_index().tail(5)
        for year, count in year_dist.items():
            pct = (count / total_docs) * 100
            print(f"  {year}: {count:,} ({pct:.1f}%)")
    
    # Top ciudades
    if 'ciudad' in final_df.columns:
        print(f"\nTop 10 ciudades:")
        city_dist = final_df['ciudad'].value_counts().head(10)
        for city, count in city_dist.items():
            pct = (count / total_docs) * 100
            print(f"  {city}: {count:,} ({pct:.1f}%)")

def find_multilingual_topics(final_df, min_languages=3):
    """Encuentra temas que aparecen en múltiples idiomas"""
    
    print(f"=== TEMAS MULTILINGÜES (mínimo {min_languages} idiomas) ===")
    
    multilingual_topics = []
    
    for sentiment in final_df['sentimiento'].unique():
        sent_data = final_df[final_df['sentimiento'] == sentiment]
        
        for topic_id in sent_data['global_topic'].unique():
            if topic_id != -1:  # Excluir outliers
                topic_data = sent_data[sent_data['global_topic'] == topic_id]
                languages = topic_data['idioma'].nunique()
                
                if languages >= min_languages:
                    lang_dist = topic_data['idioma'].value_counts()
                    
                    multilingual_topics.append({
                        'sentiment': sentiment,
                        'topic_id': topic_id,
                        'total_docs': len(topic_data),
                        'num_languages': languages,
                        'languages': dict(lang_dist),
                        'dominant_language': lang_dist.index[0],
                        'language_diversity': languages / len(lang_dist)
                    })
    
    # Ordenar por diversidad lingüística
    multilingual_topics.sort(key=lambda x: x['num_languages'], reverse=True)
    
    print(f"Encontrados {len(multilingual_topics)} temas multilingües:")
    
    for i, topic in enumerate(multilingual_topics[:10], 1):  # Top 10
        print(f"\n{i}. Tema {topic['topic_id']} ({topic['sentiment']})")
        print(f"   Documentos: {topic['total_docs']:,}")
        print(f"   Idiomas: {topic['num_languages']}")
        print(f"   Distribución: {topic['languages']}")
        print(f"   Idioma dominante: {topic['dominant_language']}")
    
    return multilingual_topics

def export_topic_analysis(final_df, lang_analysis, output_dir="E:/bertopic_800k_multilingue"):
    """Exporta análisis detallado a archivos"""
    
    output_path = Path(output_dir)
    
    # Crear resumen por sentimiento
    sentiment_summary = {}
    
    for sentiment in final_df['sentimiento'].unique():
        sent_data = final_df[final_df['sentimiento'] == sentiment]
        
        # Top 20 temas para este sentimiento
        top_topics = sent_data['global_topic'].value_counts().head(20)
        
        topic_details = []
        for topic_id, count in top_topics.items():
            if topic_id != -1:
                topic_docs = sent_data[sent_data['global_topic'] == topic_id]
                
                topic_info = {
                    'topic_id': int(topic_id),
                    'document_count': int(count),
                    'percentage': round((count / len(sent_data)) * 100, 2),
                    'languages': topic_docs['idioma'].value_counts().to_dict(),
                    'language_count': topic_docs['idioma'].nunique(),
                    'dominant_language': topic_docs['idioma'].value_counts().index[0],
                    'years': topic_docs['año'].value_counts().to_dict() if 'año' in topic_docs.columns else {},
                    'cities': topic_docs['ciudad'].value_counts().head(5).to_dict() if 'ciudad' in topic_docs.columns else {},
                    'categories': topic_docs['categoria'].value_counts().head(3).to_dict() if 'categoria' in topic_docs.columns else {},
                    'sample_texts': topic_docs['texto'].head(3).tolist()
                }
                topic_details.append(topic_info)
        
        sentiment_summary[sentiment] = {
            'total_documents': len(sent_data),
            'total_topics': sent_data['global_topic'].nunique() - (1 if -1 in sent_data['global_topic'].values else 0),
            'outliers': len(sent_data[sent_data['global_topic'] == -1]),
            'unique_languages': sent_data['idioma'].nunique(),
            'language_distribution': sent_data['idioma'].value_counts().to_dict(),
            'top_topics': topic_details
        }
    
    # Guardar análisis
    with open(output_path / "detailed_topic_analysis.json", 'w', encoding='utf-8') as f:
        json.dump(sentiment_summary, f, indent=2, ensure_ascii=False)
    
    print(f"Análisis detallado guardado en: {output_path / 'detailed_topic_analysis.json'}")
    
    return sentiment_summary

## Informe impreso
- Generan tus funciones analyze_all_sentiments + analyze_multilingual_topics. Resume, por sentimiento (negativo / neutro / positivo), los Top temas (global_topic) y sus métricas.
- Dashboard textual que te dice, por sentimiento, cuáles son los temas más grandes, en qué idiomas aparecen, en qué años/ciudades se concentran y ejemplos reales.

In [30]:
analyze_all_sentiments(final_df, lang_analysis, top_n=10)

=== ANÁLISIS COMPLETO DE TODOS LOS SENTIMIENTOS ===
Sentimientos encontrados: ['negativo', 'neutro', 'positivo']

=== ANÁLISIS DE TEMAS MULTILINGÜES (NEGATIVO) ===
Total documentos para negativo: 101,666

Top 10 temas más frecuentes:

1. Tema 2: 24,381 documentos (24.0%)
   Idiomas: {'es': 23681, 'en': 470, 'pt': 56, 'nl': 35, 'it': 19}
   Diversidad: 26 idiomas, principal: es (97.1%)
   Años principales: {2024.0: 11136, 2025.0: 7303, 2023.0: 5361}
   Ciudades principales: {'Barcelona': 11818, 'Madrid': 5460, 'Malaga': 4584}
   Categorías: {'Tour': 10965, 'Desconocido': 4431, 'Vida nocturna': 3642}
   Ejemplos de documentos:
      1. [es] - Barcelona (2024.0): Esta entrada no tiene comentarios
      2. [es] - Barcelona (2024.0): Esta entrada no tiene comentarios
      3. [es] - Barcelona (2024.0): Esta entrada no tiene comentarios

3. Tema 0: 5,785 documentos (5.7%)
   Idiomas: {'en': 2483, 'es': 2237, 'fr': 404, 'de': 263, 'it': 245}
   Diversidad: 30 idiomas, principal: en (42.9%)
  

## Resumen: 
- Explica en bloques, cómo quedó el corpus tras combinar todos los pickles y remapear temas:
- Fotografía global del dataset
- Distribución por sentimiento
- Top 10 idiomas
- Top 10 ciudades

In [32]:
create_topic_summary_report(final_df, lang_analysis)

=== REPORTE RESUMEN DE TEMAS ===
Total documentos: 852,566
Total temas identificados: 218
Documentos sin clasificar (outliers): 89,023 (10.4%)

Distribución por sentimiento:
             Temas  Documentos  Idiomas
sentimiento                            
negativo       218      101666       42
neutro         151      424614       41
positivo       129      326286       45

Top 10 idiomas:
  en: 518,428 (60.8%)
  es: 276,150 (32.4%)
  fr: 19,348 (2.3%)
  ca: 12,601 (1.5%)
  it: 6,988 (0.8%)
  de: 5,539 (0.6%)
  pt: 3,400 (0.4%)
  nl: 2,973 (0.3%)
  pl: 1,419 (0.2%)
  ko: 580 (0.1%)

Distribución temporal:
  2022.0: 95,822 (11.2%)
  2023.0: 236,452 (27.7%)
  2024.0: 275,482 (32.3%)
  2025.0: 154,716 (18.1%)
  2026.0: 114 (0.0%)

Top 10 ciudades:
  Barcelona: 208,192 (24.4%)
  Madrid: 174,844 (20.5%)
  Sevilla: 99,564 (11.7%)
  Tenerife: 84,601 (9.9%)
  Malaga: 83,350 (9.8%)
  Gran Canaria: 72,647 (8.5%)
  Valencia: 65,341 (7.7%)
  Mallorca: 64,027 (7.5%)


## TEMAS MULTILINGÜES
Es un ranking de topics que están presentes en varios idiomas, con su tamaño y mezcla lingüística. Si quieres subir o bajar la exigencia, cambia el parámetro min_languages (p. ej., find_multilingual_topics(final_df, min_languages=5)).

- (mínimo 3 idiomas)”: filtro aplicado (solo topics que aparecen en ≥ 3 idiomas).
- Idiomas: cuántos idiomas distintos tiene ese topic.
- Distribución: conteo por idioma dentro del topic (diccionario idioma→#docs).
- Idioma dominante: el idioma con más documentos en ese topic.

In [34]:
multilingual_topics = find_multilingual_topics(final_df, min_languages=3)

=== TEMAS MULTILINGÜES (mínimo 3 idiomas) ===
Encontrados 408 temas multilingües:

1. Tema 8 (positivo)
   Documentos: 51,958
   Idiomas: 43
   Distribución: {'es': 26239, 'en': 17398, 'nl': 1306, 'pt': 1045, 'fr': 1043, 'pl': 745, 'it': 666, 'de': 519, 'ko': 434, 'ca': 380, 'ro': 208, 'hu': 206, 'ru': 189, 'sv': 167, 'da': 139, 'cs': 138, 'zh-tw': 121, 'no': 108, 'af': 95, 'tr': 83, 'el': 80, 'ja': 71, 'fi': 67, 'sk': 66, 'lt': 56, 'hr': 50, 'bg': 49, 'sl': 44, 'zh-cn': 41, 'uk': 35, 'et': 28, 'lv': 26, 'ar': 20, 'tl': 16, 'th': 15, 'id': 14, 'so': 13, 'mk': 9, 'sw': 8, 'cy': 7, 'he': 7, 'vi': 6, 'sq': 1}
   Idioma dominante: es

2. Tema 9 (positivo)
   Documentos: 25,684
   Idiomas: 32
   Distribución: {'es': 13856, 'en': 9523, 'fr': 919, 'it': 592, 'de': 438, 'pt': 91, 'nl': 79, 'pl': 29, 'hu': 17, 'ru': 14, 'sv': 12, 'fi': 12, 'ro': 12, 'da': 10, 'el': 9, 'sl': 9, 'cs': 9, 'no': 9, 'ko': 8, 'ca': 6, 'af': 6, 'ja': 4, 'sk': 4, 'hr': 3, 'et': 3, 'zh-cn': 2, 'zh-tw': 2, 'tl': 2, 'bg':

## 📦 Dependencias
- Importa pandas, numpy para el flujo de tópicos.
- Generador de datasets para un dashboard en Streamlit a partir de tu final_df (datos combinados) y lang_analysis. Crea archivos JSON/CSV/PKL listos para cargar en Streamlit (más un config.json con qué hay disponible).
- Produce (carpeta: E:/bertopic_800k_multilingue/streamlit_data)
- Temporal
- Geográfico
- Multilingüe
- Clima si existe

In [60]:
import pandas as pd
import json
import numpy as np
from pathlib import Path
from collections import defaultdict
import pickle

class StreamlitDataGenerator:
    """Genera datos estructurados y optimizados para Streamlit"""
    
    def __init__(self, final_df, lang_analysis, output_dir="E:/bertopic_800k_multilingue"):
        self.final_df = final_df
        self.lang_analysis = lang_analysis
        self.output_dir = Path(output_dir)
        self.streamlit_dir = self.output_dir / "streamlit_data"
        self.streamlit_dir.mkdir(exist_ok=True)
        
        print(f"Preparando datos para Streamlit en: {self.streamlit_dir}")
    
    def generate_overview_metrics(self):
        """Genera métricas generales para dashboard"""
        
        metrics = {
            'total_documents': len(self.final_df),
            'total_topics': self.final_df['global_topic'].nunique() - (1 if -1 in self.final_df['global_topic'].values else 0),
            'total_languages': self.final_df['idioma'].nunique(),
            'total_cities': self.final_df['ciudad'].nunique() if 'ciudad' in self.final_df.columns else 0,
            'total_categories': self.final_df['categoria'].nunique() if 'categoria' in self.final_df.columns else 0,
            'date_range': {
                'start': str(self.final_df['fecha'].min()) if 'fecha' in self.final_df.columns else None,
                'end': str(self.final_df['fecha'].max()) if 'fecha' in self.final_df.columns else None
            },
            'outliers_count': len(self.final_df[self.final_df['global_topic'] == -1]),
            'outliers_percentage': round(len(self.final_df[self.final_df['global_topic'] == -1]) / len(self.final_df) * 100, 2)
        }
        
        # Guardar métricas
        with open(self.streamlit_dir / "overview_metrics.json", 'w') as f:
            json.dump(metrics, f, indent=2)
        
        print(f"Métricas generales guardadas: {len(self.final_df):,} documentos, {metrics['total_topics']} temas")
        return metrics
    
    def generate_sentiment_analysis(self):
        """Genera análisis detallado por sentimiento"""
        
        sentiment_data = {}
        
        for sentiment in self.final_df['sentimiento'].unique():
            sent_data = self.final_df[self.final_df['sentimiento'] == sentiment]
            
            # Análisis de temas para este sentimiento
            topic_analysis = []
            topic_counts = sent_data['global_topic'].value_counts()
            
            for topic_id, count in topic_counts.items():
                if topic_id != -1:  # Excluir outliers
                    topic_docs = sent_data[sent_data['global_topic'] == topic_id]
                    
                    # Análisis de idiomas
                    lang_dist = topic_docs['idioma'].value_counts()
                    
                    # Análisis temporal
                    temporal_dist = {}
                    if 'año' in topic_docs.columns:
                        temporal_dist = topic_docs['año'].value_counts().to_dict()
                    
                    # Análisis geográfico
                    geo_dist = {}
                    if 'ciudad' in topic_docs.columns:
                        geo_dist = topic_docs['ciudad'].value_counts().head(10).to_dict()
                    
                    # Análisis categórico
                    cat_dist = {}
                    if 'categoria' in topic_docs.columns:
                        cat_dist = topic_docs['categoria'].value_counts().head(5).to_dict()
                    
                    # Ejemplos representativos
                    samples = []
                    sample_docs = topic_docs.sample(min(5, len(topic_docs)))
                    
                    for _, doc in sample_docs.iterrows():
                        samples.append({
                            'texto': doc['texto'][:200] + "..." if len(doc['texto']) > 200 else doc['texto'],
                            'idioma': doc['idioma'],
                            'ciudad': doc.get('ciudad', 'N/A'),
                            'categoria': doc.get('categoria', 'N/A'),
                            'fecha': str(doc.get('fecha', 'N/A')),
                            'confianza': doc.get('confianza', 'N/A')
                        })
                    
                    topic_analysis.append({
                        'topic_id': int(topic_id),
                        'document_count': int(count),
                        'percentage': round((count / len(sent_data)) * 100, 2),
                        'language_diversity': lang_dist.nunique(),
                        'dominant_language': lang_dist.index[0] if len(lang_dist) > 0 else 'unknown',
                        'language_distribution': lang_dist.to_dict(),
                        'temporal_distribution': temporal_dist,
                        'geographic_distribution': geo_dist,
                        'category_distribution': cat_dist,
                        'sample_documents': samples,
                        'multilingual': lang_dist.nunique() >= 3
                    })
            
            # Ordenar por número de documentos
            topic_analysis.sort(key=lambda x: x['document_count'], reverse=True)
            
            sentiment_data[sentiment] = {
                'total_documents': len(sent_data),
                'total_topics': len([t for t in topic_analysis]),
                'outliers': len(sent_data[sent_data['global_topic'] == -1]),
                'language_summary': sent_data['idioma'].value_counts().to_dict(),
                'temporal_summary': sent_data['año'].value_counts().to_dict() if 'año' in sent_data.columns else {},
                'geographic_summary': sent_data['ciudad'].value_counts().head(20).to_dict() if 'ciudad' in sent_data.columns else {},
                'category_summary': sent_data['categoria'].value_counts().to_dict() if 'categoria' in sent_data.columns else {},
                'topics': topic_analysis
            }
        
        # Guardar análisis por sentimiento
        with open(self.streamlit_dir / "sentiment_analysis.json", 'w', encoding='utf-8') as f:
            json.dump(sentiment_data, f, indent=2, ensure_ascii=False)
        
        print(f"Análisis por sentimiento guardado: {len(sentiment_data)} sentimientos")
        return sentiment_data
    
    def generate_language_analysis(self):
        """Genera análisis detallado por idioma"""
        
        language_data = {}
        
        for language in self.final_df['idioma'].unique():
            lang_data = self.final_df[self.final_df['idioma'] == language]
            
            # Análisis por sentimiento para este idioma
            sentiment_breakdown = {}
            for sentiment in lang_data['sentimiento'].unique():
                sent_lang_data = lang_data[lang_data['sentimiento'] == sentiment]
                
                # Top temas para este idioma-sentimiento
                top_topics = sent_lang_data['global_topic'].value_counts().head(10)
                topic_list = []
                
                for topic_id, count in top_topics.items():
                    if topic_id != -1:
                        topic_list.append({
                            'topic_id': int(topic_id),
                            'document_count': int(count),
                            'percentage': round((count / len(sent_lang_data)) * 100, 2)
                        })
                
                sentiment_breakdown[sentiment] = {
                    'document_count': len(sent_lang_data),
                    'topic_count': sent_lang_data['global_topic'].nunique() - (1 if -1 in sent_lang_data['global_topic'].values else 0),
                    'top_topics': topic_list
                }
            
            language_data[language] = {
                'total_documents': len(lang_data),
                'document_percentage': round((len(lang_data) / len(self.final_df)) * 100, 2),
                'unique_topics': lang_data['global_topic'].nunique() - (1 if -1 in lang_data['global_topic'].values else 0),
                'sentiment_breakdown': sentiment_breakdown,
                'geographic_distribution': lang_data['ciudad'].value_counts().head(10).to_dict() if 'ciudad' in lang_data.columns else {},
                'temporal_distribution': lang_data['año'].value_counts().to_dict() if 'año' in lang_data.columns else {},
                'category_distribution': lang_data['categoria'].value_counts().head(5).to_dict() if 'categoria' in lang_data.columns else {}
            }
        
        # Ordenar por número de documentos
        language_data = dict(sorted(language_data.items(), 
                                  key=lambda x: x[1]['total_documents'], 
                                  reverse=True))
        
        # Guardar análisis por idioma
        with open(self.streamlit_dir / "language_analysis.json", 'w', encoding='utf-8') as f:
            json.dump(language_data, f, indent=2, ensure_ascii=False)
        
        print(f"Análisis por idioma guardado: {len(language_data)} idiomas")
        return language_data
    
    def generate_topic_explorer_data(self):
        """Genera datos para explorador interactivo de temas"""
        
        topic_explorer = []
        
        for sentiment in self.final_df['sentimiento'].unique():
            sent_data = self.final_df[self.final_df['sentimiento'] == sentiment]
            
            for topic_id in sent_data['global_topic'].unique():
                if topic_id != -1:
                    topic_docs = sent_data[sent_data['global_topic'] == topic_id]
                    
                    # Estadísticas del tema
                    lang_stats = topic_docs['idioma'].value_counts()
                    
                    topic_explorer.append({
                        'sentiment': sentiment,
                        'topic_id': int(topic_id),
                        'document_count': len(topic_docs),
                        'percentage_in_sentiment': round((len(topic_docs) / len(sent_data)) * 100, 2),
                        'language_count': lang_stats.nunique(),
                        'dominant_language': lang_stats.index[0] if len(lang_stats) > 0 else 'unknown',
                        'languages': lang_stats.to_dict(),
                        'is_multilingual': lang_stats.nunique() >= 3,
                        'top_cities': topic_docs['ciudad'].value_counts().head(5).to_dict() if 'ciudad' in topic_docs.columns else {},
                        'top_categories': topic_docs['categoria'].value_counts().head(3).to_dict() if 'categoria' in topic_docs.columns else {},
                        'year_range': {
                            'start': int(topic_docs['año'].min()) if 'año' in topic_docs.columns and topic_docs['año'].notna().any() else None,
                            'end': int(topic_docs['año'].max()) if 'año' in topic_docs.columns and topic_docs['año'].notna().any() else None
                        },
                        'sample_text': topic_docs['texto'].iloc[0][:300] + "..." if len(topic_docs['texto'].iloc[0]) > 300 else topic_docs['texto'].iloc[0]
                    })
        
        # Ordenar por número de documentos
        topic_explorer.sort(key=lambda x: x['document_count'], reverse=True)
        
        # Convertir a DataFrame para Streamlit
        topic_df = pd.DataFrame(topic_explorer)
        topic_df.to_csv(self.streamlit_dir / "topic_explorer.csv", index=False, encoding='utf-8')
        topic_df.to_pickle(self.streamlit_dir / "topic_explorer.pkl")
        
        print(f"Explorador de temas guardado: {len(topic_explorer)} temas")
        return topic_df
    
    def generate_time_series_data(self):
        """Genera datos para análisis temporal - VERSIÓN CORREGIDA para datetime64[ns]"""
        
        if 'fecha' not in self.final_df.columns:
            print("No hay columna 'fecha' disponible")
            return None
        
        # Filtrar datos con fechas válidas (no nulas)
        valid_dates = self.final_df[self.final_df['fecha'].notna()].copy()
        
        if len(valid_dates) == 0:
            print("No hay fechas válidas en los datos")
            return None
        
        print(f"Procesando {len(valid_dates):,} registros con fechas válidas de {len(self.final_df):,} totales")
        
        # Extraer componentes de fecha directamente desde datetime64[ns]
        valid_dates['año_extraido'] = valid_dates['fecha'].dt.year
        valid_dates['mes_extraido'] = valid_dates['fecha'].dt.month
        valid_dates['dia_extraido'] = valid_dates['fecha'].dt.day
        
        # Crear year_month string para agrupación
        valid_dates['year_month'] = valid_dates['fecha'].dt.strftime('%Y-%m')
        
        # Análisis temporal por sentimiento
        time_series = []
        
        for sentiment in valid_dates['sentimiento'].unique():
            sent_data = valid_dates[valid_dates['sentimiento'] == sentiment]
            
            # Agrupación temporal por año y mes usando los componentes extraídos
            temporal_groups = sent_data.groupby(['año_extraido', 'mes_extraido']).agg({
                'global_topic': 'nunique',
                'idioma': 'nunique',
                'texto': 'count',
                'ciudad': 'nunique'
            }).reset_index()
            
            temporal_groups.columns = ['año', 'mes', 'unique_topics', 'unique_languages', 'document_count', 'unique_cities']
            temporal_groups['sentiment'] = sentiment
            
            # CORRECCIÓN: Crear fecha usando pd.date_range o construcción manual
            temporal_groups['date'] = pd.to_datetime({
                'year': temporal_groups['año'],
                'month': temporal_groups['mes'], 
                'day': 1  # Usar día 1 para todos
            })
            temporal_groups['year_month'] = temporal_groups['date'].dt.strftime('%Y-%m')
            
            time_series.append(temporal_groups)
        
        if time_series:
            time_df = pd.concat(time_series, ignore_index=True)
            
            # Ordenar por fecha
            time_df = time_df.sort_values(['año', 'mes'])
            
            # Agregar estadísticas adicionales
            time_df['outliers_count'] = 0
            time_df['outliers_percentage'] = 0.0
            
            # Calcular outliers por período de manera más eficiente
            for sentiment in valid_dates['sentimiento'].unique():
                sent_data = valid_dates[valid_dates['sentimiento'] == sentiment]
                
                # Agrupar outliers por año-mes
                outliers_by_period = sent_data[sent_data['global_topic'] == -1].groupby(['año_extraido', 'mes_extraido']).size()
                
                # Actualizar DataFrame principal
                for (año, mes), outlier_count in outliers_by_period.items():
                    mask = (time_df['sentiment'] == sentiment) & (time_df['año'] == año) & (time_df['mes'] == mes)
                    time_df.loc[mask, 'outliers_count'] = outlier_count
            
            # Calcular porcentajes
            time_df['outliers_percentage'] = (time_df['outliers_count'] / time_df['document_count'] * 100).round(2)
            
            # Guardar datos temporales
            time_df.to_csv(self.streamlit_dir / "time_series.csv", index=False)
            time_df.to_pickle(self.streamlit_dir / "time_series.pkl")
            
            # Crear resumen mensual agregado (todos los sentimientos)
            monthly_summary = valid_dates.groupby(['año_extraido', 'mes_extraido']).agg({
                'global_topic': 'nunique',
                'idioma': 'nunique',
                'sentimiento': 'nunique',
                'texto': 'count',
                'ciudad': 'nunique'
            }).reset_index()
            
            monthly_summary.columns = ['año', 'mes', 'unique_topics', 'unique_languages', 'unique_sentiments', 'document_count', 'unique_cities']
            
            # CORRECCIÓN: Misma construcción de fecha para resumen mensual
            monthly_summary['date'] = pd.to_datetime({
                'year': monthly_summary['año'],
                'month': monthly_summary['mes'],
                'day': 1
            })
            monthly_summary['year_month'] = monthly_summary['date'].dt.strftime('%Y-%m')
            
            monthly_summary.to_csv(self.streamlit_dir / "monthly_summary.csv", index=False)
            
            print(f"Datos temporales guardados: {len(time_df)} puntos de tiempo")
            print(f"Rango temporal: {time_df['año'].min()}-{time_df['mes'].min():02d} a {time_df['año'].max()}-{time_df['mes'].max():02d}")
            
            return time_df
        
        return None
    def generate_daily_trends(self):
        """Genera tendencias diarias - VERSIÓN CORREGIDA"""
        
        if 'fecha' not in self.final_df.columns:
            return None
        
        # Filtrar datos válidos
        valid_dates = self.final_df[self.final_df['fecha'].notna()].copy()
        
        if len(valid_dates) == 0:
            return None
        
        try:
            # Extraer solo la fecha (sin hora) para agrupación diaria
            valid_dates['date_only'] = valid_dates['fecha'].dt.date
            
            # Análisis diario (solo si hay suficientes datos)
            daily_counts = valid_dates['date_only'].value_counts()
            
            if len(daily_counts) > 30:  # Solo si tenemos más de 30 días de datos
                daily_trends = []
                
                for sentiment in valid_dates['sentimiento'].unique():
                    sent_data = valid_dates[valid_dates['sentimiento'] == sentiment]
                    
                    daily_sent = sent_data.groupby('date_only').agg({
                        'texto': 'count',
                        'global_topic': 'nunique',
                        'idioma': 'nunique'
                    }).reset_index()
                    
                    daily_sent.columns = ['date', 'document_count', 'unique_topics', 'unique_languages']
                    daily_sent['sentiment'] = sentiment
                    daily_sent['date_str'] = daily_sent['date'].astype(str)
                    
                    # Convertir date a datetime para ordenamiento
                    daily_sent['date_dt'] = pd.to_datetime(daily_sent['date'])
                    
                    daily_trends.append(daily_sent)
                
                if daily_trends:
                    daily_df = pd.concat(daily_trends, ignore_index=True)
                    daily_df = daily_df.sort_values('date_dt')
                    
                    # Remover la columna auxiliar antes de guardar
                    daily_df = daily_df.drop('date_dt', axis=1)
                    
                    daily_df.to_csv(self.streamlit_dir / "daily_trends.csv", index=False)
                    
                    print(f"Tendencias diarias guardadas: {len(daily_df)} puntos diarios")
                    return daily_df
        
        except Exception as e:
            print(f"Error generando tendencias diarias: {e}")
        
        return None
    def generate_enhanced_temporal_analysis(self):
        """Análisis temporal mejorado - VERSIÓN CORREGIDA"""
        
        if 'fecha' not in self.final_df.columns:
            print("No hay datos de fecha disponibles")
            return None
        
        # Verificar datos válidos
        valid_count = self.final_df['fecha'].notna().sum()
        total_count = len(self.final_df)
        
        print(f"Fechas válidas: {valid_count:,} de {total_count:,} ({valid_count/total_count*100:.1f}%)")
        
        if valid_count == 0:
            print("No hay fechas válidas para análisis temporal")
            return None
        
        # Generar series temporales principales
        time_series = self.generate_time_series_data()
        
        # Generar tendencias diarias si hay suficientes datos
        daily_trends = self.generate_daily_trends()
        
        # Análisis de estacionalidad
        valid_dates = self.final_df[self.final_df['fecha'].notna()].copy()
        
        if len(valid_dates) > 0:
            try:
                # Extraer componentes temporales
                valid_dates['month'] = valid_dates['fecha'].dt.month
                valid_dates['weekday'] = valid_dates['fecha'].dt.dayofweek
                valid_dates['quarter'] = valid_dates['fecha'].dt.quarter
                valid_dates['year'] = valid_dates['fecha'].dt.year
                
                # Análisis estacional
                seasonal_analysis = {}
                
                # Por mes
                month_sentiment = valid_dates.groupby(['month', 'sentimiento']).size().unstack(fill_value=0)
                seasonal_analysis['by_month'] = month_sentiment.to_dict('index')
                
                # Por día de la semana
                weekday_sentiment = valid_dates.groupby(['weekday', 'sentimiento']).size().unstack(fill_value=0)
                seasonal_analysis['by_weekday'] = weekday_sentiment.to_dict('index')
                
                # Por trimestre
                quarter_sentiment = valid_dates.groupby(['quarter', 'sentimiento']).size().unstack(fill_value=0)
                seasonal_analysis['by_quarter'] = quarter_sentiment.to_dict('index')
                
                # Por año
                year_sentiment = valid_dates.groupby(['year', 'sentimiento']).size().unstack(fill_value=0)
                seasonal_analysis['by_year'] = year_sentiment.to_dict('index')
                
                with open(self.streamlit_dir / "seasonal_analysis.json", 'w') as f:
                    json.dump(seasonal_analysis, f, indent=2, default=str)
                
                print("Análisis estacional guardado")
                
            except Exception as e:
                print(f"Error en análisis estacional: {e}")
        
        return {
            'time_series': time_series,
            'daily_trends': daily_trends,
            'seasonal_available': True,
            'valid_dates_count': valid_count,
            'total_dates_count': total_count
        }
    def generate_geographic_data(self):
        """Genera datos para análisis geográfico"""
        
        if 'ciudad' not in self.final_df.columns:
            print("No hay datos geográficos disponibles")
            return None
        
        geo_data = []
        
        for city in self.final_df['ciudad'].value_counts().head(50).index:  # Top 50 ciudades
            city_data = self.final_df[self.final_df['ciudad'] == city]
            
            sentiment_breakdown = city_data['sentimiento'].value_counts().to_dict()
            language_breakdown = city_data['idioma'].value_counts().to_dict()
            
            geo_data.append({
                'ciudad': city,
                'total_documents': len(city_data),
                'unique_topics': city_data['global_topic'].nunique() - (1 if -1 in city_data['global_topic'].values else 0),
                'unique_languages': city_data['idioma'].nunique(),
                'sentiment_breakdown': sentiment_breakdown,
                'language_breakdown': language_breakdown,
                'dominant_sentiment': max(sentiment_breakdown, key=sentiment_breakdown.get),
                'dominant_language': max(language_breakdown, key=language_breakdown.get)
            })
        
        geo_df = pd.DataFrame(geo_data)
        geo_df.to_csv(self.streamlit_dir / "geographic_data.csv", index=False, encoding='utf-8')
        geo_df.to_pickle(self.streamlit_dir / "geographic_data.pkl")
        
        print(f"Datos geográficos guardados: {len(geo_df)} ciudades")
        return geo_df
    
    def generate_multilingual_insights(self):
        """Genera insights específicos para temas multilingües"""
        
        multilingual_topics = []
        
        for sentiment in self.final_df['sentimiento'].unique():
            sent_data = self.final_df[self.final_df['sentimiento'] == sentiment]
            
            for topic_id in sent_data['global_topic'].unique():
                if topic_id != -1:
                    topic_docs = sent_data[sent_data['global_topic'] == topic_id]
                    language_count = topic_docs['idioma'].nunique()
                    
                    if language_count >= 3:  # Temas con 3+ idiomas
                        lang_dist = topic_docs['idioma'].value_counts()
                        
                        # Calcular diversidad lingüística (índice de Shannon)
                        total_docs = len(topic_docs)
                        shannon_diversity = -sum((count/total_docs) * np.log(count/total_docs) 
                                               for count in lang_dist.values)
                        
                        multilingual_topics.append({
                            'sentiment': sentiment,
                            'topic_id': int(topic_id),
                            'document_count': total_docs,
                            'language_count': language_count,
                            'shannon_diversity': round(shannon_diversity, 3),
                            'languages': lang_dist.to_dict(),
                            'dominant_language': lang_dist.index[0],
                            'dominance_percentage': round((lang_dist.iloc[0] / total_docs) * 100, 2),
                            'geographic_spread': topic_docs['ciudad'].nunique() if 'ciudad' in topic_docs.columns else 0,
                            'category_spread': topic_docs['categoria'].nunique() if 'categoria' in topic_docs.columns else 0
                        })
        
        # Ordenar por diversidad lingüística
        multilingual_topics.sort(key=lambda x: x['shannon_diversity'], reverse=True)
        
        # Guardar insights multilingües
        with open(self.streamlit_dir / "multilingual_insights.json", 'w', encoding='utf-8') as f:
            json.dump(multilingual_topics, f, indent=2, ensure_ascii=False)
        
        multilingual_df = pd.DataFrame(multilingual_topics)
        if len(multilingual_df) > 0:
            multilingual_df.to_csv(self.streamlit_dir / "multilingual_topics.csv", index=False, encoding='utf-8')
            multilingual_df.to_pickle(self.streamlit_dir / "multilingual_topics.pkl")
        
        print(f"Insights multilingües guardados: {len(multilingual_topics)} temas multilingües")
        return multilingual_topics
    def generate_weather_sentiment_analysis(self):
        """Genera análisis de correlación entre clima y sentimiento"""
        
        if 'descripcion_sencilla' not in self.final_df.columns:
            print("No hay datos climáticos disponibles")
            return None
        
        # Filtrar datos con información climática válida
        weather_data = self.final_df.dropna(subset=['descripcion_sencilla'])
        
        if len(weather_data) == 0:
            print("No hay descripciones climáticas válidas")
            return None
        
        weather_analysis = {}
        
        # Análisis por condición climática
        for weather in weather_data['descripcion_sencilla'].unique():
            weather_subset = weather_data[weather_data['descripcion_sencilla'] == weather]
            
            # Distribución de sentimientos por clima
            sentiment_dist = weather_subset['sentimiento'].value_counts()
            
            # Análisis de temas por clima
            topic_analysis = {}
            for sentiment in weather_subset['sentimiento'].unique():
                sent_weather_data = weather_subset[weather_subset['sentimiento'] == sentiment]
                top_topics = sent_weather_data['global_topic'].value_counts().head(5)
                
                topic_list = []
                for topic_id, count in top_topics.items():
                    if topic_id != -1:
                        topic_list.append({
                            'topic_id': int(topic_id),
                            'document_count': int(count),
                            'percentage': round((count / len(sent_weather_data)) * 100, 2)
                        })
                
                topic_analysis[sentiment] = {
                    'document_count': len(sent_weather_data),
                    'top_topics': topic_list
                }
            
            # Análisis temporal por clima
            temporal_dist = {}
            if 'fecha' in weather_subset.columns:
                weather_subset_copy = weather_subset.copy()
                weather_subset_copy['fecha_parsed'] = pd.to_datetime(weather_subset_copy['fecha'])
                weather_subset_copy['month'] = weather_subset_copy['fecha_parsed'].dt.month
                temporal_dist = weather_subset_copy['month'].value_counts().to_dict()
            
            weather_analysis[weather] = {
                'total_documents': len(weather_subset),
                'percentage_of_total': round((len(weather_subset) / len(weather_data)) * 100, 2),
                'sentiment_distribution': sentiment_dist.to_dict(),
                'dominant_sentiment': sentiment_dist.index[0] if len(sentiment_dist) > 0 else 'unknown',
                'sentiment_percentage': round((sentiment_dist.iloc[0] / len(weather_subset)) * 100, 2) if len(sentiment_dist) > 0 else 0,
                'topic_analysis_by_sentiment': topic_analysis,
                'monthly_distribution': temporal_dist,
                'unique_languages': weather_subset['idioma'].nunique(),
                'unique_cities': weather_subset['ciudad'].nunique() if 'ciudad' in weather_subset.columns else 0
            }
        
        # Ordenar por frecuencia
        weather_analysis = dict(sorted(weather_analysis.items(), 
                                     key=lambda x: x[1]['total_documents'], 
                                     reverse=True))
        
        # Guardar análisis climático
        with open(self.streamlit_dir / "weather_sentiment_analysis.json", 'w', encoding='utf-8') as f:
            json.dump(weather_analysis, f, indent=2, ensure_ascii=False)
        
        print(f"Análisis climático guardado: {len(weather_analysis)} condiciones climáticas")
        return weather_analysis

    def generate_weather_temporal_correlation(self):
        """Correlaciones clima-tiempo - VERSIÓN CORREGIDA para fechas con nulos"""
        
        if 'descripcion_sencilla' not in self.final_df.columns or 'fecha' not in self.final_df.columns:
            print("Datos insuficientes para correlación climático-temporal")
            return None
        
        # Filtrar datos válidos (clima y fecha no nulos)
        valid_data = self.final_df[
            (self.final_df['descripcion_sencilla'].notna()) & 
            (self.final_df['fecha'].notna())
        ].copy()
        
        if len(valid_data) == 0:
            print("No hay datos válidos para correlación climático-temporal")
            return None
        
        print(f"Datos válidos para análisis climático-temporal: {len(valid_data):,}")
        
        try:
            # Extraer componentes temporales
            valid_data['month'] = valid_data['fecha'].dt.month
            valid_data['year'] = valid_data['fecha'].dt.year
            valid_data['season'] = valid_data['month'].map({
                12: 'Invierno', 1: 'Invierno', 2: 'Invierno',
                3: 'Primavera', 4: 'Primavera', 5: 'Primavera',
                6: 'Verano', 7: 'Verano', 8: 'Verano',
                9: 'Otoño', 10: 'Otoño', 11: 'Otoño'
            })
            
            # Análisis estacional del clima
            seasonal_weather = []
            
            for season in valid_data['season'].unique():
                season_data = valid_data[valid_data['season'] == season]
                
                for weather in season_data['descripcion_sencilla'].unique():
                    weather_season_data = season_data[season_data['descripcion_sencilla'] == weather]
                    
                    if len(weather_season_data) > 0:  # Verificar que hay datos
                        sentiment_dist = weather_season_data['sentimiento'].value_counts()
                        
                        seasonal_weather.append({
                            'season': season,
                            'weather': weather,
                            'total_documents': len(weather_season_data),
                            'sentiment_breakdown': sentiment_dist.to_dict(),
                            'dominant_sentiment': sentiment_dist.index[0] if len(sentiment_dist) > 0 else 'unknown',
                            'unique_topics': weather_season_data['global_topic'].nunique() - (1 if -1 in weather_season_data['global_topic'].values else 0),
                            'unique_languages': weather_season_data['idioma'].nunique(),
                            'unique_cities': weather_season_data['ciudad'].nunique() if 'ciudad' in weather_season_data.columns else 0
                        })
            
            if seasonal_weather:
                # Convertir a DataFrame para análisis
                seasonal_df = pd.DataFrame(seasonal_weather)
                seasonal_df.to_csv(self.streamlit_dir / "seasonal_weather_correlation.csv", index=False, encoding='utf-8')
                
                # Crear matriz de correlación clima-sentimiento
                correlation_matrix = valid_data.groupby(['descripcion_sencilla', 'sentimiento']).size().unstack(fill_value=0)
                correlation_matrix.to_csv(self.streamlit_dir / "weather_sentiment_matrix.csv", encoding='utf-8')
                
                print(f"Correlación climático-temporal guardada: {len(seasonal_weather)} combinaciones estación-clima")
                return seasonal_df
            else:
                print("No se generaron datos de correlación estacional")
                return None
            
        except Exception as e:
            print(f"Error en correlación climático-temporal: {e}")
            return None

    def generate_enhanced_topic_analysis_with_weather(self):
        """Genera análisis de temas enriquecido con datos climáticos"""
        
        if 'descripcion_sencilla' not in self.final_df.columns:
            print("No hay datos climáticos para enriquecer análisis de temas")
            return None
        
        weather_topic_analysis = {}
        
        for sentiment in self.final_df['sentimiento'].unique():
            sent_data = self.final_df[self.final_df['sentimiento'] == sentiment]
            
            topic_weather_analysis = []
            
            # Analizar top 20 temas para este sentimiento
            top_topics = sent_data['global_topic'].value_counts().head(20)
            
            for topic_id, count in top_topics.items():
                if topic_id != -1:
                    topic_data = sent_data[sent_data['global_topic'] == topic_id]
                    
                    # Análisis climático para este tema
                    weather_dist = topic_data['descripcion_sencilla'].value_counts()
                    
                    # Análisis temporal-climático
                    temporal_weather = {}
                    if 'fecha' in topic_data.columns:
                        try:
                            topic_data_copy = topic_data.copy()
                            topic_data_copy['fecha_parsed'] = pd.to_datetime(topic_data_copy['fecha'])
                            topic_data_copy['month'] = topic_data_copy['fecha_parsed'].dt.month
                            
                            for month in topic_data_copy['month'].unique():
                                month_data = topic_data_copy[topic_data_copy['month'] == month]
                                month_weather = month_data['descripcion_sencilla'].value_counts()
                                temporal_weather[int(month)] = month_weather.to_dict()
                        except:
                            pass
                    
                    # Ejemplos con contexto climático
                    weather_examples = {}
                    for weather in weather_dist.head(3).index:
                        weather_docs = topic_data[topic_data['descripcion_sencilla'] == weather]
                        sample = weather_docs.sample(min(2, len(weather_docs)))
                        
                        examples = []
                        for _, doc in sample.iterrows():
                            examples.append({
                                'texto': doc['texto'][:150] + "..." if len(doc['texto']) > 150 else doc['texto'],
                                'idioma': doc['idioma'],
                                'ciudad': doc.get('ciudad', 'N/A'),
                                'fecha': str(doc.get('fecha', 'N/A'))
                            })
                        
                        weather_examples[weather] = examples
                    
                    topic_weather_analysis.append({
                        'topic_id': int(topic_id),
                        'document_count': int(count),
                        'percentage': round((count / len(sent_data)) * 100, 2),
                        'weather_distribution': weather_dist.to_dict(),
                        'dominant_weather': weather_dist.index[0] if len(weather_dist) > 0 else 'unknown',
                        'weather_diversity': weather_dist.nunique(),
                        'temporal_weather_patterns': temporal_weather,
                        'weather_examples': weather_examples,
                        'weather_sentiment_coherence': self._calculate_weather_coherence(weather_dist, sentiment)
                    })
            
            weather_topic_analysis[sentiment] = topic_weather_analysis
        
        # Guardar análisis enriquecido
        with open(self.streamlit_dir / "topics_with_weather_analysis.json", 'w', encoding='utf-8') as f:
            json.dump(weather_topic_analysis, f, indent=2, ensure_ascii=False)
        
        print(f"Análisis de temas con clima guardado para {len(weather_topic_analysis)} sentimientos")
        return weather_topic_analysis

    def _calculate_weather_coherence(self, weather_dist, sentiment):
        """Calcula coherencia entre clima y sentimiento"""
        
        # Mapeo simple de coherencia clima-sentimiento
        positive_weather = ['soleado', 'despejado', 'parcialmente nublado', 'claro']
        negative_weather = ['lluvioso', 'tormentoso', 'nublado', 'nevando']
        
        if len(weather_dist) == 0:
            return 0
        
        dominant_weather = weather_dist.index[0].lower()
        
        if sentiment == 'positivo':
            return 1 if any(pw in dominant_weather for pw in positive_weather) else 0
        elif sentiment == 'negativo':
            return 1 if any(nw in dominant_weather for nw in negative_weather) else 0
        else:
            return 0.5  # neutro
    
    def generate_weather_insights_summary(self):
        """Genera resumen de insights climáticos"""
        
        if 'descripcion_sencilla' not in self.final_df.columns:
            return None
        
        weather_data = self.final_df.dropna(subset=['descripcion_sencilla'])
        
        insights = {
            'total_documents_with_weather': len(weather_data),
            'unique_weather_conditions': weather_data['descripcion_sencilla'].nunique(),
            'weather_frequency': weather_data['descripcion_sencilla'].value_counts().to_dict(),
            'sentiment_weather_correlation': {},
            'weather_language_patterns': {},
            'seasonal_weather_summary': {}
        }
        
        # Correlación sentimiento-clima
        for sentiment in weather_data['sentimiento'].unique():
            sent_weather = weather_data[weather_data['sentimiento'] == sentiment]
            insights['sentiment_weather_correlation'][sentiment] = {
                'most_common_weather': sent_weather['descripcion_sencilla'].value_counts().head(5).to_dict(),
                'document_count': len(sent_weather)
            }
        
        # Patrones idioma-clima
        for weather in weather_data['descripcion_sencilla'].value_counts().head(10).index:
            weather_subset = weather_data[weather_data['descripcion_sencilla'] == weather]
            insights['weather_language_patterns'][weather] = {
                'languages': weather_subset['idioma'].value_counts().head(5).to_dict(),
                'dominant_language': weather_subset['idioma'].value_counts().index[0]
            }
        
        # Guardar insights
        with open(self.streamlit_dir / "weather_insights_summary.json", 'w', encoding='utf-8') as f:
            json.dump(insights, f, indent=2, ensure_ascii=False)
        
        print(f"Resumen de insights climáticos guardado")
        return insights
    def generate_all_streamlit_data_with_weather(self):
        """Genera todos los datasets incluyendo análisis climático completo - VERSIÓN CORREGIDA"""
        
        print("=== GENERANDO DATOS PARA STREAMLIT (CON ANÁLISIS CLIMÁTICO) ===")
        
        results = {}
        
        # Análisis básicos existentes
        try:
            results['overview'] = self.generate_overview_metrics()
            print("✓ Métricas generales completadas")
        except Exception as e:
            print(f"✗ Error en métricas generales: {e}")
            results['overview'] = None
        
        try:
            results['sentiment_analysis'] = self.generate_sentiment_analysis()
            print("✓ Análisis por sentimiento completado")
        except Exception as e:
            print(f"✗ Error en análisis por sentimiento: {e}")
            results['sentiment_analysis'] = None
        
        try:
            results['language_analysis'] = self.generate_language_analysis()
            print("✓ Análisis por idioma completado")
        except Exception as e:
            print(f"✗ Error en análisis por idioma: {e}")
            results['language_analysis'] = None
        
        try:
            results['topic_explorer'] = self.generate_topic_explorer_data()
            print("✓ Explorador de temas completado")
        except Exception as e:
            print(f"✗ Error en explorador de temas: {e}")
            results['topic_explorer'] = None
        
        # Análisis temporal mejorado
        try:
            results['temporal_analysis'] = self.generate_enhanced_temporal_analysis()
            if results['temporal_analysis']:
                print("✓ Análisis temporal completado")
            else:
                print("- Análisis temporal no disponible")
        except Exception as e:
            print(f"✗ Error en análisis temporal: {e}")
            results['temporal_analysis'] = None
        
        # Análisis geográfico
        try:
            results['geographic'] = self.generate_geographic_data()
            if results['geographic'] is not None:
                print("✓ Datos geográficos completados")
            else:
                print("- Datos geográficos no disponibles")
        except Exception as e:
            print(f"✗ Error en datos geográficos: {e}")
            results['geographic'] = None
        
        # Análisis climáticos
        try:
            results['weather_sentiment'] = self.generate_weather_sentiment_analysis()
            if results['weather_sentiment']:
                print("✓ Análisis climático-sentimiento completado")
            else:
                print("- Análisis climático no disponible")
        except Exception as e:
            print(f"✗ Error en análisis climático: {e}")
            results['weather_sentiment'] = None
        
        try:
            results['weather_temporal'] = self.generate_weather_temporal_correlation()
            if results['weather_temporal'] is not None:
                print("✓ Correlación climático-temporal completada")
            else:
                print("- Correlación climático-temporal no disponible")
        except Exception as e:
            print(f"✗ Error en correlación climático-temporal: {e}")
            results['weather_temporal'] = None
        
        try:
            results['topics_with_weather'] = self.generate_enhanced_topic_analysis_with_weather()
            if results['topics_with_weather']:
                print("✓ Análisis de temas con clima completado")
            else:
                print("- Análisis de temas con clima no disponible")
        except Exception as e:
            print(f"✗ Error en análisis de temas con clima: {e}")
            results['topics_with_weather'] = None
        
        try:
            results['weather_insights'] = self.generate_weather_insights_summary()
            if results['weather_insights']:
                print("✓ Resumen de insights climáticos completado")
            else:
                print("- Insights climáticos no disponibles")
        except Exception as e:
            print(f"✗ Error en insights climáticos: {e}")
            results['weather_insights'] = None
        
        try:
            results['multilingual'] = self.generate_multilingual_insights()
            print("✓ Insights multilingües completados")
        except Exception as e:
            print(f"✗ Error en insights multilingües: {e}")
            results['multilingual'] = None
        
        # CORRECCIÓN: Crear configuración con validación de tipos apropiada
        available_datasets = []
        
        # Función auxiliar para verificar si un resultado es válido
        def is_valid_result(result):
            if result is None:
                return False
            if isinstance(result, dict) and len(result) == 0:
                return False
            if isinstance(result, list) and len(result) == 0:
                return False
            if hasattr(result, '__len__') and len(result) == 0:
                return False
            return True
        
        # Verificar cada dataset individualmente
        if is_valid_result(results.get('overview')):
            available_datasets.append('overview_metrics.json')
        
        if is_valid_result(results.get('sentiment_analysis')):
            available_datasets.append('sentiment_analysis.json')
        
        if is_valid_result(results.get('language_analysis')):
            available_datasets.append('language_analysis.json')
        
        if is_valid_result(results.get('topic_explorer')):
            available_datasets.append('topic_explorer.csv')
        
        if is_valid_result(results.get('temporal_analysis')):
            available_datasets.append('time_series.csv')
            available_datasets.append('monthly_summary.csv')
        
        if is_valid_result(results.get('geographic')):
            available_datasets.append('geographic_data.csv')
        
        if is_valid_result(results.get('weather_sentiment')):
            available_datasets.append('weather_sentiment_analysis.json')
        
        if is_valid_result(results.get('weather_temporal')):
            available_datasets.append('seasonal_weather_correlation.csv')
            available_datasets.append('weather_sentiment_matrix.csv')
        
        if is_valid_result(results.get('topics_with_weather')):
            available_datasets.append('topics_with_weather_analysis.json')
        
        if is_valid_result(results.get('weather_insights')):
            available_datasets.append('weather_insights_summary.json')
        
        if is_valid_result(results.get('multilingual')):
            available_datasets.append('multilingual_insights.json')
        
        # Crear configuración
        config = {
            'data_path': str(self.streamlit_dir),
            'total_documents': len(self.final_df),
            'total_topics': 0,
            'sentiments': list(self.final_df['sentimiento'].unique()),
            'languages': list(self.final_df['idioma'].unique()[:20]),
            'weather_conditions': list(self.final_df['descripcion_sencilla'].unique()) if 'descripcion_sencilla' in self.final_df.columns else [],
            'available_datasets': available_datasets,
            'weather_analysis_available': is_valid_result(results.get('weather_sentiment')),
            'temporal_analysis_available': is_valid_result(results.get('temporal_analysis')),
            'geographic_analysis_available': is_valid_result(results.get('geographic')),
            'generation_timestamp': pd.Timestamp.now().isoformat(),
            'datasets_generated': {
                'overview': is_valid_result(results.get('overview')),
                'sentiment_analysis': is_valid_result(results.get('sentiment_analysis')),
                'language_analysis': is_valid_result(results.get('language_analysis')),
                'topic_explorer': is_valid_result(results.get('topic_explorer')),
                'temporal_analysis': is_valid_result(results.get('temporal_analysis')),
                'geographic': is_valid_result(results.get('geographic')),
                'weather_sentiment': is_valid_result(results.get('weather_sentiment')),
                'weather_temporal': is_valid_result(results.get('weather_temporal')),
                'topics_with_weather': is_valid_result(results.get('topics_with_weather')),
                'weather_insights': is_valid_result(results.get('weather_insights')),
                'multilingual': is_valid_result(results.get('multilingual'))
            }
        }
        
        # Obtener total de temas si hay datos de overview válidos
        if is_valid_result(results.get('overview')):
            config['total_topics'] = results['overview'].get('total_topics', 0)
        
        with open(self.streamlit_dir / "config.json", 'w') as f:
            json.dump(config, f, indent=2)
        
        # Resumen final
        successful_datasets = sum(config['datasets_generated'].values())
        total_datasets = len(config['datasets_generated'])
        
        print(f"\nDATOS CON ANÁLISIS CLIMÁTICO GENERADOS")
        print(f"Directorio: {self.streamlit_dir}")
        print(f"Datasets exitosos: {successful_datasets}/{total_datasets}")
        print(f"Archivos disponibles: {len(available_datasets)}")
        print(f"Análisis climático: {'Sí' if config['weather_analysis_available'] else 'No'}")
        print(f"Análisis temporal: {'Sí' if config['temporal_analysis_available'] else 'No'}")
        print(f"Análisis geográfico: {'Sí' if config['geographic_analysis_available'] else 'No'}")
        
        return results


def create_streamlit_data(final_df, lang_analysis, output_dir="E:/bertopic_800k_multilingue"):
    """Función principal para generar datos de Streamlit"""
    
    generator = StreamlitDataGenerator(final_df, lang_analysis, output_dir)
    results = generator.generate_all_streamlit_data_with_weather()
    
    return results

## ⚙️ Configuración de BERTopic
- Define hiperparámetros/pipe (vectorizador, umap/hdbscan, idioma).

In [62]:
streamlit_results = create_streamlit_data(final_df, lang_analysis, "E:/bertopic_800k_multilingue")

Preparando datos para Streamlit en: E:\bertopic_800k_multilingue\streamlit_data
=== GENERANDO DATOS PARA STREAMLIT (CON ANÁLISIS CLIMÁTICO) ===
Métricas generales guardadas: 852,566 documentos, 218 temas
✓ Métricas generales completadas
Análisis por sentimiento guardado: 3 sentimientos
✓ Análisis por sentimiento completado
Análisis por idioma guardado: 45 idiomas
✓ Análisis por idioma completado
Explorador de temas guardado: 498 temas
✓ Explorador de temas completado
Fechas válidas: 807,124 de 852,566 (94.7%)
Procesando 807,124 registros con fechas válidas de 852,566 totales
Datos temporales guardados: 489 puntos de tiempo
Rango temporal: 2009-01 a 2026-12
Tendencias diarias guardadas: 5278 puntos diarios
Análisis estacional guardado
✓ Análisis temporal completado
Datos geográficos guardados: 8 ciudades
✓ Datos geográficos completados
Análisis climático guardado: 35 condiciones climáticas
✓ Análisis climático-sentimiento completado
Datos válidos para análisis climático-temporal: 701,63